In [1]:
import pandas as pd
import cobra
from collections import OrderedDict
from copy import deepcopy
import os,sys
model_root_path = '../../../'
sys.path.append(model_root_path+'pycore/')
from gsm_custom_functions import *

In [2]:
# Metabolic model (COBRApy json)
model = cobra.io.load_json_model('../../../build_model/input/iRhtoC.json')

# Protein
df_pro = pd.read_excel('../../model/PROTEIN_stoich_curation.tsv')
df_pro.index = df_pro.id.to_list()
df_aamap = pd.read_csv('../../input/PROTEIN_amino_acid_map.txt', sep='\t')
df_aamap.index = df_aamap.aa_abbv.to_list()

# Dummy protein
df_aa_dummy = pd.read_excel('../../input/PROTEIN_dummy_prot_calc.xlsx')
df_aa_dummy.index = df_aa_dummy.aa_abbv.to_list()
dummy_medianL = int(round(df_aa_dummy.loc['A', 'Unnamed: 5'], 0))
# Add 1e-5 g/mmol to prevent number round lost of protein MW
dummy_MW = round(df_aa_dummy.loc['C', 'Unnamed: 5'], 5) + 1e-5

# Enzyme
df_enz = pd.read_excel('../../model/ENZYME_stoich_curation.tsv')

# RNA
df_rnas = pd.read_excel('../../input/RNA_stoich.xlsx')
df_rnas.index = df_rnas.RNAid.to_list()

# Ribosome
df_ribo_nuc = pd.read_excel('../../input/RIBOSOME_nucleus.xlsx')
df_ribo_nuc = df_ribo_nuc[df_ribo_nuc.paralog.isnull()]

df_ribo_mit = pd.read_excel('../../input/RIBOSOME_mitochondria.xlsx')
df_ribo_mit = df_ribo_mit[df_ribo_mit.paralog.isnull()]

# Biomass
df_biom = pd.read_excel('../../input/BIOMASS_RBA.xlsx', sheet_name='RBABioRxns')

In [3]:
fva = cobra.flux_analysis.flux_variability_analysis(model)
# find max biomass flux
biomass = "BIOMASS"
biomass_rxn = model.reactions.get_by_id(biomass)
biomass_fva = cobra.flux_analysis.flux_variability_analysis(model, [biomass])
biomass_max = biomass_fva.loc[biomass, 'maximum']

In [50]:
# updates to metabolite formulas
newmodel = deepcopy(model)
for met in newmodel.metabolites:
    if 'Cer' in met.formula:
        print("Old:"+str(met.elements))
        # remove polyprenyl from formula and add C90H144
        met.formula = adjust_formula(met.formula, 'C86H174N2O7Cer-1')
        print("New:"+str(met.elements))
# save new model
cobra.io.save_json_model(newmodel, '../../input/optional/iRhtoC-temp.json')

Old:{'Ce': 1}
New:{'C': 86, 'H': 174, 'N': 2, 'O': 7}
Old:{'Ce': 1}
New:{'C': 86, 'H': 174, 'N': 2, 'O': 7}
Old:{'Ce': 1}
New:{'C': 86, 'H': 174, 'N': 2, 'O': 7}
Old:{'C': 6, 'H': 10, 'O': 8, 'P': 1, 'Ce': 1}
New:{'C': 92, 'H': 184, 'O': 15, 'P': 1, 'N': 2}
Old:{'C': 6, 'H': 10, 'O': 8, 'P': 1, 'Ce': 1}
New:{'C': 92, 'H': 184, 'O': 15, 'P': 1, 'N': 2}
Old:{'C': 6, 'H': 10, 'O': 8, 'P': 1, 'Ce': 1}
New:{'C': 92, 'H': 184, 'O': 15, 'P': 1, 'N': 2}
Old:{'C': 12, 'H': 20, 'O': 13, 'P': 1, 'Ce': 1}
New:{'C': 98, 'H': 194, 'O': 20, 'P': 1, 'N': 2}
Old:{'C': 12, 'H': 20, 'O': 13, 'P': 1, 'Ce': 1}
New:{'C': 98, 'H': 194, 'O': 20, 'P': 1, 'N': 2}
Old:{'C': 12, 'H': 20, 'O': 13, 'P': 1, 'Ce': 1}
New:{'C': 98, 'H': 194, 'O': 20, 'P': 1, 'N': 2}
Old:{'C': 18, 'H': 30, 'O': 21, 'P': 2, 'Ce': 1}
New:{'C': 104, 'H': 204, 'O': 28, 'P': 2, 'N': 2}
Old:{'C': 18, 'H': 30, 'O': 21, 'P': 2, 'Ce': 1}
New:{'C': 104, 'H': 204, 'O': 28, 'P': 2, 'N': 2}
Old:{'C': 18, 'H': 30, 'O': 21, 'P': 2, 'Ce': 1}
New:{'C':

In [10]:
# Make version of model with no generic cofactors, for publication and for MEMOTE testing
model_no_generics = deepcopy(model)
# remove rxns with names beginning with "GENERIC_"
for rxn in model.reactions:
    if rxn.id.startswith("GENERIC_"):
        # print(rxn.id)
        model_no_generics.reactions.get_by_id(rxn.id).remove_from_model()
report_mass_balance(model_no_generics)

compACYLCOA_c
0.01875 docoscoa_c + 0.355664 linocoa_c + 0.087612 linolncoa_c + 0.207129 odecoa_c + 0.1694 pmtcoa_c + 0.112806 stcoa_c + 0.048639 ttccoa_c <=> acylcoa_c
acylcoa_c	C21H31N7O16P3SAcyl	-4	C00040	Acyl-CoA	manual
docoscoa_c	C43H74N7O17P3S1	-4	C16528	docosanoyl-CoA	biocyc
linocoa_c	C39H62N7O17P3S	-4	C02050	linoleoyl-CoA	seed.compound
linolncoa_c	C39H60N7O17P3S	-4	C16162	alpha-linolenoyl-CoA	seed.compound
odecoa_c	C39H64N7O17P3S1	-4	C00510	oleoyl-CoA	biocyc
pmtcoa_c	C37H62N7O17P3S1	-4	C00154	palmitoyl-CoA	biocyc
stcoa_c	C39H66N7O17P3S1	-4	C00412	stearoyl-CoA	biocyc
ttccoa_c	C45H78N7O17P3S	-4	C16529	tetracosanoyl-CoA	seed.compound
C:-18.028033999999998, H:-32.693481999999996, N:3.3306690738754696e-16, O:-0.999999999999999, P:5.551115123125783e-17, S:-5.551115123125783e-17, Acyl:1, charge:2.220446049250313e-16

compACYLCOA_l
0.01875 docoscoa_l + 0.355664 linocoa_l + 0.087612 linolncoa_l + 0.207129 odecoa_l + 0.1694 pmtcoa_l + 0.112806 stcoa_l + 0.048639 ttccoa_l <=> acylcoa_l
acy

(['ACACT180i_x',
  'ACOAO260_x',
  '3OACR200_rm',
  '3HACD260_rm',
  'ECOAR200_rm',
  'ECOAR260_rm',
  'FACOAL260_rm',
  'ACOAO200_x',
  'ACOAO161b_x',
  'ACOAO181a_x',
  'ACOAO161a_x',
  'ECOAH260_x',
  'ECOAH200_x',
  'ECOAH240_x',
  'HACD180i_x',
  'HACD240i_x',
  'HACD141bi_x',
  'HACD121ai_x',
  'HACD181ai_x',
  'HACD161ai_x',
  'HACD141ai_x',
  'ACACT181ai_x',
  'ECOAI101a_x',
  'ECOAI121a_x',
  'HXCCOAt_c_l',
  'HXCCOAt_c_en',
  'FACOAL260_x',
  'ACACT180i_m',
  'ACACT181ai_m',
  'HACD180i_m',
  'HACD121ai_m',
  'HACD141ai_m',
  'HACD141bi_m',
  'HACD161ai_m',
  'HACD181ai_m',
  'CERS126_c',
  'CERS2A26_r',
  'ACOAD161bf_x',
  'ACOAD181af_x',
  'ACOAD161af_x',
  'HLYSMT_m'],
 [],
 ['compACYLCOA_c',
  'compACYLCOA_l',
  'compACYLCOA_m',
  'compACYLCOA_r',
  'compACYLCOA_rm',
  'compACYLCOA_x',
  'compFALPD_mm',
  'compFALPD_m',
  'compFALPD_rm',
  'compFALPD_en',
  'BIOMASS',
  'DM_5mta_c',
  'compFALPD_c',
  'DM_gcald_c'])

In [13]:
newmodel = cobra.io.load_json_model('../../input/optional/iRhtoC-temp.json')
# newmodel = deepcopy(model)
newmodel.solver

In [14]:
fva_new = cobra.flux_analysis.flux_variability_analysis(newmodel)

In [15]:
biomass_rxn = newmodel.reactions.get_by_id(biomass)
biomass_fva = cobra.flux_analysis.flux_variability_analysis(newmodel, [biomass])
biomass_max_new = biomass_fva.loc[biomass, 'maximum']

In [16]:
# compare FVA results
tol = max(model.tolerance, newmodel.tolerance)
fva_compare = pd.DataFrame()
fva_compare['old_min'] = fva['minimum']
fva_compare['old_max'] = fva['maximum']
fva_compare['new_min'] = fva_new['minimum']
fva_compare['new_max'] = fva_new['maximum']
fva_compare['diff_min'] = fva_compare['old_min'] - fva_compare['new_min']
fva_compare['diff_max'] = fva_compare['old_max'] - fva_compare['new_max']
# show entries with non-negligible differences
display(fva_compare[(abs(fva_compare['diff_min']) > tol) | (abs(fva_compare['diff_max']) > tol)])
# output to csv for easy comparison
fva_compare.to_csv('./fva_compare.csv')

,old_min,old_max,new_min,new_max,diff_min,diff_max
13GS_c,0.390627,0.390627,0.390312,0.390312,3.146457e-04,3.146457e-04
16GS_c,0.102797,0.102797,0.102714,0.102714,8.280169e-05,8.280169e-05
PRMICI_c,0.029772,0.029772,0.029748,0.029748,2.398117e-05,2.398117e-05
ACHBS_m,0.090859,0.090859,0.090786,0.090786,7.318622e-05,7.318613e-05
AATA_c,0.101349,0.101349,0.101267,0.101267,8.163535e-05,8.163526e-05
...,...,...,...,...,...,...
RNDR4_c,0.000000,0.002768,0.000000,0.002766,0.000000e+00,2.229852e-06
PKETX_c,2.125679,2.125679,2.133764,2.133764,-8.085450e-03,-8.085474e-03
ACS_c,2.152209,2.152209,2.160273,2.160273,-8.064080e-03,-8.064105e-03
AKGDH_m,0.000312,0.000312,0.000312,0.000312,2.514342e-07,2.488387e-07


In [17]:
# MW of ipc_g in old and new models
ipc_g = newmodel.metabolites.get_by_id('ipc_g')
display(ipc_g.formula_weight)
ipc_g_old = model.metabolites.get_by_id('ipc_g')
display(ipc_g_old.formula_weight)
# find MW of "C6H10O8P"
ipc_g_new = cobra.Metabolite('ipc_g', formula='C6H10O8P', name='Inositolphosphoceramide (IPC) (g)', compartment='g')
display(ipc_g_new.formula_weight)

1589.4235210000002

381.228561

241.112561

In [36]:
fva_compare[(abs(fva_compare['diff_min']) > 1e-10) | (abs(fva_compare['diff_max']) > 1e-10)]

,old_min,old_max,new_min,new_max,diff_min,diff_max
COAt_c_rm,-0.118974,-0.027005,-0.118974,-0.027005,-2.583350e-12,-3.909300e-10
